In [ ]:
# use data cornll05
# TODO change datareader
# train model
# 

In [61]:
- [x] pull out the last hidden states and make it a sentence embedding
- [x] read in image features
- [x] choose a score funciton between image feature and sentence embedding
- [x] integrate the score function into loss function\
- [x] text to instance funciton need to be defined
- [x] give model parameter in config file, leave others to allennlp config
- [x] train
- [ ] To be decided

found for 18177 out of 30190 tokens


SyntaxError: invalid syntax (<ipython-input-61-3794fa5ffebd>, line 1)

In [46]:
import numpy as np

TODO:
- [x] Change datareader to cornll05 train set formmat
- [x]  run model for test:
- [x] change model to accept json train test data
- [x] invalid sentence problem, some sentence does not have verb in it.
- [x] test model
- [ ] adding image attenter into the model
    * image to conv features? easiest way is to use plain attention
    * attention layer
    * plain encoder: image, decoder, text
    
- [ ] see performance 
- [ ] make some results:
    - [ ] adding parsing
- [ ] transformer
- [ ] different alignment
- [ ] 
Intuition

By trying to align image with sentence
the sentence representation learning part would learn gain more information during optimzing, and 
the sentence representation acquired will make be grounded to the image
Thus, intrinsic knowledge is embeded into the representation, and when testing, even without the image part, model can do inference with "image knowledge in mind", and idealy do it better.

m to overturn the
long-held belief that syntactic parsing is a prerequisite for this task (Punyakanok et al., 2008).

g Zhou and Xu (2015), we treat SRL as a
BIO tagging problem 

introducing highway connections (Srivastava et al., 2015; Zhang
et al., 2016), (3) using recurrent dropout (Gal
and Ghahramani, 2016), (4) decoding with BIOconstraints, and (5) ensembling with a product of
experts

1) design choices on architecture, initialization, and
regularization that have a surprisingly large impact on model performance; (2) different types of
prediction errors showing, e.g., that deep models
excel at predicting long-distance dependencies but
still struggle with known challenges such as **PPattachment errors and adjunct-argument distinctions; **

cross entropy with sequence


# Change the datareader to cornll05

In [40]:
# ls /media/data/srl/srl/

In [28]:
ontonotes_datapath = '/media/data/srl/srl/conll-formatted-ontonotes-5.0/'

In [22]:
import os
from allennlp.data.dataset_readers import SrlReader

In [49]:
import numpy as np

In [12]:
from allennlp.data.token_indexers import SingleIdTokenIndexer, TokenIndexer
from allennlp.data.fields import Field, TextField, SequenceLabelField, MetadataField
from allennlp.data.instance import Instance
from pytorch_pretrained_bert.tokenization import BertTokenizer
from allennlp.data.tokenizers import Token
import logging
from typing import Dict, List, Iterable, Tuple, Any
from allennlp.common.file_utils import cached_path

In [19]:
from allennlp.data.dataset_readers import SrlReader
from allennlp.models import srl_bert
import json

In [32]:
# train_reader = SrlReader(lazy=True, domain_identifier='train')

# train_data = train_reader.read(ontonotes_datapath)

# test_reader = SrlReader(lazy=True, domain_identifier='test')
# test_data = test_reader.read(ontonotes_datapath)

# val_reader = SrlReader(lazy=True, domain_identifier='development')
# val_data = val_reader.read(ontonotes_datapath)

# logger = logging.getLogger(__name__)

In [36]:
def _read(file_path):
    file_path = cached_path(file_path)
    logger.info(f"Reading SRL instances fro dataset files at:{file_path}")
    with open(file_path) as f:
        for line in f.readlines():
            srl = json.loads(line)
            tokens = [Token(t) for t in srl["words"]]
            tags = srl['verbs'][0]['tags']
            verb_indictor = [1 if label[-2:] == "-V" else 0 for label in tags]
            yield self.tokens, verb_indictor, tags
            

In [39]:
class ImageSrlReader(SrlReader):
    def _read(self, file_path):
        file_path = cached_path(file_path)
        logger.info(f"Reading SRL instances fro dataset files at:{file_path}")
        with open(file_path) as f:
            for line in f.readlines():
                srl = json.loads(line)
                tokens = [Token(t) for t in srl["words"]]
                tags = srl['verbs'][0]['tags']
                verb_indictor = [1 if label[-2:] == "-V" else 0 for label in tags]
                yield self.text_to_instance(tokens, verb_indictor, tags)


In [41]:
ls

'Create dataset.ipynb'   output/                 srl_label_getter.ipynb
 experiments/            README.md               train_new.json
 LICENSE                 srl_json_reader.ipynb


In [55]:
file_path = '/home/zijiao/work/data/mscoco/'

In [56]:
ls ~/work/data/mscoco

dev_caps.txt  test_caps.txt          test_new.json   train_ims.npy
dev_ims.npy   test_ground-truth.txt  train_caps.txt  vocab.pkl


In [57]:
def load_img(file_path, data_split):
    return np.load(os.path.join(file_path, f'{data_split}_ims.npy'))

In [58]:
val_image = load_img(file_path, 'dev')

In [59]:
val_image.shape

(1000, 2048)

In [60]:
type(val_image)

numpy.ndarray

In [46]:
import numpy as np